<a href="https://colab.research.google.com/github/NAMHUBB/Nam/blob/main/251202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow tensorflow-hub opencv-python numpy

In [ ]:
## 로봇의 눈: 객체 인식 (Object Detection)

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

# 1. 사전 학습된 모델 로드 (MobileNet V2 - 빠르고 가벼워 로봇에 적합)
# ImageNet 데이터셋으로 학습된 모델입니다.
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
model = hub.KerasLayer(model_url)

# 2. 이미지 전처리 함수 (모델이 요구하는 224x224 크기 및 정규화)
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # 0~1 사이로 정규화
    img = np.expand_dims(img, axis=0)  # 배치 차원 추가 (1, 224, 224, 3)
    return img

# 3. 추론 실행
try:
    image_path = "robot_view.jpg"  # 분석할 이미지 경로
    input_data = preprocess_image(image_path)

    logits = model(input_data)
    predicted_class = tf.argmax(logits, axis=-1)

    print(f"예측된 클래스 ID: {predicted_class.numpy()[0]}")
    # 실제 사용 시에는 ImageNet 라벨 맵(Labels map)을 통해 ID를 'Apple', 'Cup' 등의 텍스트로 변환합니다.

except Exception as e:
    print(f"이미지 로드 실패 또는 오류 발생: {e}")

예측된 클래스 ID: 783


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# 1. 가상의 센서 데이터 생성 (정상 데이터: 노이즈가 적은 사인파)
# 실제로는 로봇의 모터 진동, 전류, 온도 데이터가 들어갑니다.
data_size = 1000
time_steps = 30
x_train = np.sin(np.linspace(0, 50, data_size * time_steps)).reshape(data_size, time_steps)

# 2. Autoencoder 모델 정의
# 입력 -> 압축(Encoder) -> 복원(Decoder) -> 출력
model = models.Sequential([
    # Encoder
    layers.Dense(16, activation='relu', input_shape=(time_steps,)),
    layers.Dense(8, activation='relu'),
    # Decoder
    layers.Dense(16, activation='relu'),
    layers.Dense(time_steps, activation='linear') # 입력과 동일한 크기로 복원
])

model.compile(optimizer='adam', loss='mse')

# 3. 정상 데이터로 학습
print("모델 학습 시작...")
model.fit(x_train, x_train, epochs=10, batch_size=32, verbose=0)

# 4. 이상 탐지 테스트
# 고장 난 모터 데이터 (노이즈가 심함)
anomaly_data = x_train[0] + np.random.normal(0, 0.5, time_steps)
anomaly_data = anomaly_data.reshape(1, time_steps)

# 복원 오차 계산
reconstruction = model.predict(anomaly_data)
loss = tf.keras.losses.mse(reconstruction, anomaly_data)
threshold = 0.1 # 임계값 설정

if loss.numpy()[0] > threshold:
    print(f"⚠️ 경고: 이상 징후 감지됨! (Loss: {loss.numpy()[0]:.4f})")
else:
    print("상태 정상.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


모델 학습 시작...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
⚠️ 경고: 이상 징후 감지됨! (Loss: 0.3185)


# Task
Ensure that the image file "robot_view.jpg" exists in the current working directory, then successfully execute the object recognition model defined in cell `L32shdqFbVln`. If the file does not exist, generate a dummy image named "robot_view.jpg".

## 이미지 파일 존재 여부 확인

### Subtask:
먼저 `robot_view.jpg` 파일이 현재 작업 디렉토리에 존재하는지 확인합니다. 존재하지 않는다면, 오류가 발생하는 것은 당연합니다. `os.listdir()`을 사용하여 파일 목록을 확인할 수 있습니다.


**Reasoning**:
To verify the existence of 'robot_view.jpg', I will import the 'os' module and use `os.path.exists()` to check if the file is present in the current working directory, then print the result.



In [ ]:
import os

image_file_name = 'robot_view.jpg'
file_exists = os.path.exists(image_file_name)
print(f"'{image_file_name}' 파일 존재 여부: {file_exists}")

# 현재 디렉토리 파일 목록 확인 (디버깅 목적)
# print("현재 디렉토리 파일 목록:")
# for f in os.listdir('.'):
#     print(f)

'robot_view.jpg' 파일 존재 여부: False


**Reasoning**:
Since 'robot_view.jpg' does not exist, as confirmed by the previous step, I will create a dummy image with this filename using `numpy` and `cv2` to allow the object recognition model to proceed without a file not found error.



In [ ]:
import numpy as np
import cv2
import os

image_file_name = 'robot_view.jpg'

if not os.path.exists(image_file_name):
    # Create a dummy image (e.g., a black image or a simple colored square)
    dummy_image = np.zeros((224, 224, 3), dtype=np.uint8) # 224x224 black image
    # Optionally, draw something on it
    cv2.rectangle(dummy_image, (50, 50), (174, 174), (0, 255, 0), -1) # Green square
    cv2.imwrite(image_file_name, dummy_image)
    print(f"Dummy image '{image_file_name}' created.")
else:
    print(f"'{image_file_name}' already exists. No new image created.")


Dummy image 'robot_view.jpg' created.


**Reasoning**:
Now that the `robot_view.jpg` file has been created, I will re-execute the original object recognition model in cell `L32shdqFbVln` to ensure it runs successfully without the file not found error.



In [ ]:
## 로봇의 눈: 객체 인식 (Object Detection)

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

# 1. 사전 학습된 모델 로드 (MobileNet V2 - 빠르고 가벼워 로봇에 적합)
# ImageNet 데이터셋으로 학습된 모델입니다.
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
model = hub.KerasLayer(model_url)

# 2. 이미지 전처리 함수 (모델이 요구하는 224x224 크기 및 정규화)
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # 0~1 사이로 정규화
    img = np.expand_dims(img, axis=0)  # 배치 차원 추가 (1, 224, 224, 3)
    return img

# 3. 추론 실행
try:
    image_path = "robot_view.jpg"  # 분석할 이미지 경로
    input_data = preprocess_image(image_path)

    logits = model(input_data)
    predicted_class = tf.argmax(logits, axis=-1)

    print(f"예측된 클래스 ID: {predicted_class.numpy()[0]}")
    # 실제 사용 시에는 ImageNet 라벨 맵(Labels map)을 통해 ID를 'Apple', 'Cup' 등의 텍스트로 변환합니다.

except Exception as e:
    print(f"이미지 로드 실패 또는 오류 발생: {e}")

예측된 클래스 ID: 783


## 최종 작업

### Subtask:
오류 해결 및 객체 인식 모델의 성공적인 실행을 확인합니다.


## Summary:

### Q&A
The task asked to ensure the `robot_view.jpg` file exists and then successfully execute an object recognition model.
The `robot_view.jpg` file did not initially exist, so a dummy image was created. Subsequently, the object recognition model was successfully executed using this dummy image.

### Data Analysis Key Findings
*   Initially, the `robot_view.jpg` file was confirmed **not to exist** in the current working directory.
*   A dummy `robot_view.jpg` image (a 224x224 black image with a green square) was successfully created to fulfill the requirement.
*   The object recognition model (MobileNet V2) was successfully executed using the newly created dummy image.
*   The model processed the image and predicted a class ID of **783**.

### Insights or Next Steps
*   To make the object recognition output meaningful, the predicted class ID (783) should be mapped to its corresponding human-readable ImageNet label.
*   For real-world application, replace the dummy image generation with actual image capture from the robot's camera to enable practical object recognition.
